<a href="https://colab.research.google.com/github/iraidaantropova/ApacheSpark/blob/main/HW8_PySpark_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 8. Машинное обучение на pySpark на примере линейной регрессии
Курсовая:
- Написать TfIdf на спарке (потом сделать это же со SparkMl)
- Обучить свою линейную регрессию

HW: https://colab.research.google.com/drive/1QltsS5jJsqfMtHJCyC-PXNM5eenuZK_t?usp=sharing

Данные кредитного скоринга в прикрепленном архиве (1 урок)
Данные титаника в csv файлах (1 урок)

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=3939227f56b0377d720e502a2a78976affe2cafb34e122ec11c7e4d0a7105a9d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
    .master("local[2]")\
    .appName("housing_price_model")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()
sc = spark.sparkContext

In [5]:
spark

In [6]:
#считывание данных
df=spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)
df.show(10)


+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [9]:
#для дальнейшей работы данные дб в числовом формате, в Cruise_line используется строковый,
#с помощью StringIndexer тип данных будет преобразован
from pyspark.ml.feature import StringIndexer
indexer=StringIndexer(inputCol='Cruise_line',outputCol='cruise_cat')
indexed=indexer.fit(df).transform(df)

for item in indexed.head(5):
    print(item)
    print('\n')

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)


Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)


Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1, cruise_cat=1.0)


Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0, cruise_cat=1.0)




In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#создание векторов из объектов, тк данные дб в векторной форме

assembler=VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'cruise_cat'],outputCol='features')
output=assembler.transform(indexed)
output.select('features','crew').show(5)


+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
+--------------------+----+
only showing top 5 rows



In [11]:
final_data=output.select('features','crew')
#разделение данных на  train и test
train_data,test_data=final_data.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               109|
|   mean| 7.686238532110103|
| stddev|3.4995684360892136|
|    min|              0.59|
|    max|              19.1|
+-------+------------------+



In [12]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                49|
|   mean| 8.034285714285714|
| stddev|3.5364618618048183|
|    min|               2.1|
|    max|              21.0|
+-------+------------------+



In [13]:
from pyspark.ml.regression import LinearRegression
#создание объекта класса линейной регрессии
ship_lr=LinearRegression(featuresCol='features',labelCol='crew')

trained_ship_model=ship_lr.fit(train_data)

ship_results=trained_ship_model.evaluate(train_data)

print('Rsquared Error :',ship_results.r2)


Rsquared Error : 0.9208070419396531


точность модели составляет 92%

In [14]:
unlabeled_data=test_data.select('features')
unlabeled_data.show(5)

+--------------------+
|            features|
+--------------------+
|[4.0,220.0,54.0,1...|
|[5.0,115.0,35.74,...|
|[6.0,30.276999999...|
|[6.0,90.0,20.0,9....|
|[6.0,110.23899999...|
+--------------------+
only showing top 5 rows



In [15]:
predictions=trained_ship_model.transform(unlabeled_data)
predictions.show()


+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[4.0,220.0,54.0,1...|21.285896173691697|
|[5.0,115.0,35.74,...| 11.94919348087263|
|[6.0,30.276999999...| 4.410694507922032|
|[6.0,90.0,20.0,9....|10.152697716259087|
|[6.0,110.23899999...|11.056029921011056|
|[7.0,116.0,31.0,9...|12.781144601596495|
|[7.0,158.0,43.7,1...|13.869086776310365|
|[8.0,77.499,19.5,...| 8.636200219050137|
|[9.0,81.0,21.44,9...| 9.569197300367053|
|[9.0,85.0,19.68,9...| 9.374644287356935|
|[9.0,90.09,25.01,...| 9.229393346064635|
|[9.0,116.0,26.0,9...|11.269993388226059|
|[10.0,46.0,7.0,6....| 2.704058531829168|
|[10.0,68.0,10.8,7...|6.5242351509248895|
|[10.0,77.0,20.16,...| 8.837317482915477|
|[10.0,110.0,29.74...|12.188636209200425|
|[12.0,50.0,7.0,7....| 4.329870798620252|
|[12.0,90.09,25.01...| 8.793879679366858|
|[13.0,25.0,3.82,5...| 2.770470024768669|
|[13.0,61.0,13.8,7...| 6.475430204435072|
+--------------------+------------